In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import csv
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
COLOR_TREAT = "#2ecc71"
COLOR_NO_TREAT = "#e74c3c"

Peace agreement analysis :
- agreement features
- agreement's success features

** PAID** Peace Agreement ID

**Region**
The regional variable specifies the regional location of the conflict:
1. Europe: Geographic definition, including the states in the Caucasus. (COW numbers 200–395) 
2. Middle East: Egypt, Iran, Iraq, Israel, Jordan, Kuwait, Lebanon, Syria, Turkey, and the states of the Arabian Peninsula (COW numbers 630–698)
3. Asia: Geographic definition, including Oceania, Australia, and New Zealand, and excluding states in the Middle East. (COW numbers 700–990)
4. Africa: Geographic definition, excluding states in the Middle East (eg. Egypt). (COW numbers 400– 625)
5. Americas: Geographic definition, including states in the Caribbean. (COW numbers 2–165)

**GWNO** Gleditsch & Ward System Membership Table

**CID** - Conflict ID

In [ ]:
pa = pd.read_excel('data/ucdp-peace-agreements.xls')

In [ ]:
pa.iloc[:10, :11]

In [ ]:
# ended - Agreement ended:
pa.iloc[:10, 9:20]

In [ ]:
reader = csv.reader(open('data/translate_conf.csv', 'r'))
d = {}
for row in reader:
    k, v = row
    d[v] = k
    
d['9']    

In [ ]:
pa.CID = pa.CID.astype(dtype=str).replace(to_replace=d )

pa.CID = pa.CID.astype(dtype = int)



## Distribution of the agreements by region

In [ ]:
sns.countplot(x='Region', data=pa)
plt.title('Agreement counts per region')
plt.xlabel('Region');
plt.xticks(range(5), ['europe', 'middle east', 'Asia', 'Africa', 'Americas']);

This plot shows the number of peace agreements per region. We can see that most agreements concern Africa.

## Distribution of agreements by year

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(x='Year', data=pa)

## Analysis of the Conflict ID variable

In [ ]:
print('Number of conflicts treated in this dataset : %s' %(len(pa.CID.unique())))

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(x='CID', data=pa, order=pa.CID.value_counts().index)

This plot shows that approximtely half of the conflicts were solved with one agreement. We wil split the rest of the exploration into two groups, the ones solved in one agreements and one grouping all the others.

In [ ]:
pa_one = pa[pa['CID'] == [10, 101, 103, 119, 124, 126, 154, 180, 186, 189, 192,194, 195, 197, 199, 203, 205, 208, 210, 214, 215
                 , 216, 218, 223, 227, 253, 250]]
pa_one

## Whatcombi : function to be ran for each new number of possible combinations 

In [ ]:
def whatcombi(pa):
    # returns the corresponding combination number of the dataset
    # set : n and rerun box in fct of the number of binary features to be analyzed
    n = 4
    lst = [list(i) for i in itertools.product([0, 1], repeat=n)]
    for i in range(len(lst)):
        a = pa == lst[i]
        # if all values in a are true, it is the good combination
        if all(x for x in a.values):
            return(i)

## Analysis of the agreements termination consistency in the variables :

- **ended** 
    - 0) if the agreements still holds
    - 1) if the agreement terminated
    
- **DyVi05** 
    - 1) the dyad violence restarted or continued
    - 0) the violence ended in this dyad
    - -99) Not applicable, agreement signed previous year

In [ ]:
pa_result = pa.loc[:, ['ended', 'DyVi05']]
pa_result.head()

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
sns.countplot(x='DyVi05', data=pa)
plt.subplot(122)
sns.countplot(x='ended', data=pa)

Thoses plots show that approximately 140 agreements ended there corresponding conflict. This information should be crossed with the number of conflicts treated in this dataset as there are only 60 unique conflicts.

In [ ]:
pa_result['combination'] = pa_result.apply(whatcombi, axis=1)

In [ ]:
sns.countplot(x='combination', data=pa_result)

Approximately 160 out of 216 (74%) agreements have the same values for the *ended* and *DyVi05* variables. Hence we can say that those variables are consistent.

## Behaviour variables analysis

In [ ]:
pa_behavior = pa.iloc[:, 20:25]
# how to analyze a set of binary features ?
pa_behavior.head()
#sns.pairplot(pa_behavior)

In [ ]:
# run only once
pa_behavior['combination'] = pa_behavior.apply(whatcombi, axis=1)

In [ ]:
sns.countplot(x='combination', data=pa_behavior)

In [ ]:
lst = [list(i) for i in itertools.product([0, 1], repeat=5)]
print(lst[11])

1. Most agreements do not treat behavioral instructions
2. The second most recurrent combination concerns agreements that provided a ceasefire
3. third most recurrent :  ceasefire, withdrawal of foreign forces
4. fourth : ceasefire, integration in national army, disarmement of warring parties

In [ ]:
pa_behavior['ended'] = pa['ended']
sns.countplot(x='combination', data= pa_behavior, hue='ended')

There are no much difference in agreements that terminate or not. However Two kinds of agreements were never terminated. The ones which provided for the creation of a new national army or the integration of rebels into the army and the withdrawal of foreign forces.

## Incompatibility variables analysis
### Government incompatibilities

In [ ]:
pa_incompatibility_gov = pa.iloc[:, 25:33]
# study ended results in fct of all combinations of the binary features
pa_incompatibility_gov.head()

In [ ]:
pa_incompatibility_gov['combination'] = pa_incompatibility_gov.apply(whatcombi, axis=1)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='combination', data=pa_incompatibility_gov)
plt.yscale('log')

In [ ]:
lst = [list(i) for i in itertools.product([0, 1], repeat=8)]
print(lst[25])

While most agreements do not deal with governmental incompatibilities, the ones that do mostly provide for elections and electoral reforms for rebel integration into the interim government.

The *Pol_prov* feature could be used to generalize the inclusion of political provisions in the agreement

## Justice variables analysis

In [ ]:
pa_justice = pa.iloc[:, 43:48]
# study ended results in fct of all combinations of the binary features
pa_justice.head()

In [ ]:
# run only once
pa_justice['combination'] = pa_justice.apply(whatcombi, axis=1)

In [ ]:
sns.countplot(x='combination', data=pa_justice)

In [ ]:
lst = [list(i) for i in itertools.product([0, 1], repeat=5)]
print(lst[9])

- The most recurring combination is the one where no justice issues is treated in the agreement.
- The second most recurring is the one where the agreement provided a release of prisoners

The last features is set to one if any of the justice provisions was applied in the agreement. This variable could be of good use to reduce the number of features and keep an insight of the rather or not 'Justice' issues treated in the agreement.

## Implementation variables analysis

In [ ]:
pa_implementation = pa.iloc[:, 48:53]
pa_implementation.drop(columns='Reaffirm ID', inplace=True)
# study ended results in fct of all combinations of the binary features
pa_implementation.head()

In [ ]:
# run only once
pa_implementation['combination'] = pa_implementation.apply(whatcombi, axis=1)

In [ ]:
sns.countplot(x='combination', data=pa_implementation)

- The most recurring implementation is the one where the agreement did not reaffirm earlier agreements, did not outline a negotiating agenda including negotiations on the incompatibility, did not provide for the deployment of a peace-keeping operation and did not provide for the establishment of a commission or committee to oversee implementation of the agreement.

- Second most recurring ... did not everything but did provide for the establishment of a commission or committee to oversee implementation of the agreement.

- Third most recurring, nothing but did outline a negotiating agenda including negotiations on the incompatibility

- Fourth : reaffirm earlier agreements and provided the establishment of a commission or committee to oversee implementation of the agreement.

## Temination variables analysis
- **DyVi05** - Violence with the same parties restarted within 5 years

    - 1) Yes, the dyad violence restarted or continued
    - 0) No, the violence ended in this dyad
    - -99) Not applicable, agreement signed previous year


- **CoVi01** - Terminated the whole conflict the following year, signed an active year
    - 2) Part of a peace process which ended the violence.
    - 1) Yes, the conflict was terminated
    - 0) No, the conflict continued
    - -99) Not applicable, for example, the peace agreement was not signed in a conflict active year.


- **noconf** - Signed in a conflict inactive year.

The peace agreement was signed in an inactive conflict year i.e. in a year with less than 25 battle- related deaths recorded by the UCDP.
    - 1) Yes 
    - 0) No

- **termdur** - Number of years since last activity.

The number of inactive years between end of conflict and signature of the peace agreement.


- **Noconf11** - Still terminated as of 2011
    No active conflict years recorded from agreement until 2011 1) Yes, no active conflict years from agreement til 2011, 
    - 0) No, conflict restarted before 2011
    - -99) Not applicable, signed in 2011
    
The *termdur* will be used to relate agreement signed on active years to ones signed on inactive year. This should allow us to give more strentgh to an agreement that was signed fewer years after the end of the conflict than one signed a long time after. 

The *DyVi05* feature will be used as the second success evaluation variable. The *CoVi01* feature will be used to evaluate the time efficiency of the agreement.

In [ ]:
pa_termination = pa.iloc[:, 53:58]
# study ended results in fct of all combinations of the binary features
pa_termination.head()

In [ ]:
pa_one = pa[pa['CID'] == [x for x in pa.CID.value_counts().index]]

In [ ]:
import copy
idx = (pa.CID.value_counts()==1)
pa_one = copy.deepcopy(pa)
for i in range(len(idx)):
    if idx.values[i] == False:
        pa_one.drop(index=[pa_one['CID'] == idx.index[i]])

In [ ]:
cc = pa.groupby(['CID', 'ended']).count()
plt.figure(figsize=(20,10))
sns.countplot(x='CID', data= pa, hue='ended')

# Conflicts

In [ ]:
df = pd.read_csv('data/ucdp-prio-acd-171.csv')

In [ ]:
df.iloc[:10, :11]

In [ ]:
df.iloc[:10, 11:27]

In [ ]:
df.drop(['sidea2nd', 'sideb2nd' , 'terr' , 'year' , 'intensity' , 'startprec' , 'startdate2' , 'epend' , 'ependprec', 'gwnoa', 'gwnoa2nd', 'gwnob', 'gwnob2nd', 'gwnoloc', 'startprec2', 'version'], axis=1, inplace = True)

In [ ]:
print('Number of unique conflicts in this dataset : %s' %(len(df.conflictid.unique())))

In [ ]:
clean_df = df.groupby('conflictid').last()

In [ ]:
clean_df

In [ ]:
agreements = pa.groupby('CID').last()
agreements.index

In [ ]:
clean_df["agreement"] = [1 if ele in agreements.index else 0 for ele in clean_df.index]
clean_df.rename({'ependdate':'enddate'} , inplace = True, axis = 1)

In [ ]:
clean_df

In [ ]:
print("Number of non-unique 'side b' for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['side b'].unique().apply(len) > 1)]))
#df.groupby(df['conflictid'])['side b'].unique()[df.groupby(df['conflictid'])['side b'].unique().apply(len) > 1]
print("Number of non-unique locations for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['location'].unique().apply(len) > 1)]))
print("Number of non-unique 'side a' for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['sidea'].unique().apply(len) > 1)]))
print("Number of non-unique startdates for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['startdate'].unique().apply(len) > 1)]))
print("Number of non-unique types for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['type'].unique().apply(len) > 1)]))
print("Number of non-unique regions for a single conflict: %d" %len(df['conflictid'].unique()[(df.groupby(df['conflictid'])['region'].unique().apply(len) > 1)]))

In [ ]:
def plot_distrib(s1, s2, title, xLabel, yLabel, ax=None):
    bins = np.histogram(s1)[1]
    sns.distplot(s1, kde=False, color=COLOR_NO_TREAT, norm_hist=True, ax=ax, bins=bins)
    sns.distplot(s2, kde=False, color=COLOR_TREAT, norm_hist=True, ax=ax, bins=bins)
    if ax is None:
        plt.title(title)
        plt.xlabel(xLabel)
        plt.ylabel(yLabel)
        plt.legend(['No agreement', 'Agreement'])
    else:
        ax.set_title(title)
        ax.set_xlabel(xLabel)
        ax.set_ylabel(yLabel)
        ax.legend(['No agreement', 'Agreement'])

In [ ]:
#f, axarr = plt.subplots(4, 1, figsize=(15, 18))
#for index, feature in enumerate(['incomp', 'cumint', 'type', 'region']):
    #ax = axarr[int(index/4)]#[index%3]
plt.figure(figsize=(12,5))
plot_distrib(s1=clean_df['incomp'][clean_df['agreement'] == 0], s2=clean_df['incomp'][clean_df['agreement'] == 1], title='incomp', xLabel = 'incomp', yLabel='Density', ax=None)

In [ ]:
plt.figure(figsize=(12,5))
plot_distrib(s1=clean_df['cumint'][clean_df['agreement'] == 0], s2=clean_df['cumint'][clean_df['agreement'] == 1], title='cumint', xLabel = 'cumint', yLabel='Density', ax=None)

In [ ]:
plt.figure(figsize=(12,5))
plot_distrib(s1=clean_df['type'][clean_df['agreement'] == 0], s2=clean_df['type'][clean_df['agreement'] == 1], title='type', xLabel = 'type', yLabel='Density', ax=None)

In [ ]:
locationNAcount = clean_df[clean_df['agreement'] == 0].groupby('location').count()['sidea']
locationNAcount = locationNAcount / len(clean_df[clean_df['agreement'] == 0])
locationNAcount.sort_values(ascending = False , inplace = True)
plt.figure(figsize=(12,5))
locationNAcount[:20].plot('bar')

In [ ]:
locationcount = clean_df[clean_df['agreement'] == 1].groupby('location').count()['sidea']
locationcount = locationcount / len(clean_df[clean_df['agreement'] == 1])
locationcount.sort_values(ascending = False , inplace = True)
plt.figure(figsize=(12,5))
locationcount[:20].plot('bar')

In [ ]:
regionNAcount = clean_df[clean_df['agreement'] == 0].groupby('region').count()['sidea']
regionNAcount = regionNAcount / len(clean_df[clean_df['agreement'] == 0])
regionNAcount.sort_values(ascending = False , inplace = True)
plt.figure(figsize=(12,5))
regionNAcount.plot('bar')

In [ ]:
regioncount = clean_df[clean_df['agreement'] == 1].groupby('region').count()['sidea']
regioncount = regioncount / len(clean_df[clean_df['agreement'] == 1])
regioncount.sort_values(ascending = False , inplace = True)
plt.figure(figsize=(12,5))
regioncount.plot('bar')

In [ ]:
clean_df[clean_df['agreement'] == 1].shape